In [1]:
import pandas as pd
from pathlib import Path
import json
from td001_processing import postprocessing_td001
from utils import round_float_cols

def run_enveloppe_processing(td001, td006, td007, td008):
    from td007_processing import merge_td007_tr_tv, postprocessing_td007
    from td008_processing import merge_td008_tr_tv, postprocessing_td008
    from td001_merge import merge_td001_dpe_id_envelope
    from td007_processing import agg_td007_to_td001_essential, agg_surface_envelope
    from td008_processing import agg_td008_to_td001_essential

    td008_raw_cols = td008.columns.tolist()
    td007_raw_cols = td007.columns.tolist()

    td001, td006, td007, td008 = merge_td001_dpe_id_envelope(td001=td001, td006=td006, td007=td007, td008=td008)

    td008 = merge_td008_tr_tv(td008)
    td008 = postprocessing_td008(td008)

    td007 = merge_td007_tr_tv(td007)
    td007 = postprocessing_td007(td007, td008)

    agg_td007 = agg_td007_to_td001_essential(td007)
    agg_td008 = agg_td008_to_td001_essential(td008)
    agg_surfaces = agg_surface_envelope(td007, td008)

    td001_enveloppe_agg = pd.concat([agg_td007, agg_td008, agg_surfaces], axis=1)

    td001_enveloppe_agg.index.name = 'td001_dpe_id'
    cols = [el for el in td008.columns if el not in td008_raw_cols + ['fen_lib_from_tv009',
                                                                      'fen_lib_from_tv021']]
    cols.append('td008_baie_id')
    td008_p = td008[cols]
    cols = [el for el in td007.columns if
            el not in td007_raw_cols + ["qualif_surf", 'surface_paroi_opaque_calc', 'surface_paroi_totale_calc_v1',
                                        'surface_paroi_totale_calc_v2']]
    cols.append('td007_paroi_opaque_id')
    td007_p = td007[cols]
    return td001_enveloppe_agg, td008_p, td007_p


def run_system_processing(td001, td006, td011, td012, td013, td014):
    from td011_td012_processing import merge_td012_tr_tv, postprocessing_td012, merge_td011_tr_tv, \
        agg_systeme_chauffage_essential
    from td013_td014_processing import merge_td013_tr_tv, postprocessing_td014, merge_td014_tr_tv, \
        agg_systeme_ecs_essential
    from td001_merge import merge_td001_dpe_id_system

    td011_raw_cols = td011.columns.tolist()
    td012_raw_cols = td012.columns.tolist()
    td013_raw_cols = td013.columns.tolist()
    td014_raw_cols = td014.columns.tolist()
    td001, td006, td011, td012, td013, td014 = merge_td001_dpe_id_system(td001, td006, td011, td012, td013, td014)
    td011 = merge_td011_tr_tv(td011)
    td012 = merge_td012_tr_tv(td012)
    td013 = merge_td013_tr_tv(td013)
    td014 = merge_td014_tr_tv(td014)

    td012 = postprocessing_td012(td012)

    cols = [el for el in td011.columns if el not in td011_raw_cols]
    cols.append('td011_installation_chauffage_id')
    td011_p = td011[cols]

    cols = [el for el in td012.columns if
            el not in td012_raw_cols + ['besoin_chauffage_infer', 'gen_ch_concat_txt_desc']]
    cols.append('td012_generateur_chauffage_id')
    td012_p = td012[cols]

    td001_sys_ch_agg = agg_systeme_chauffage_essential(td001, td011, td012)

    td014 = postprocessing_td014(td013, td014)

    cols = [el for el in td013.columns if el not in td013_raw_cols]
    cols.append('td013_installation_ecs_id')
    td013_p = td013[cols]

    cols = [el for el in td014.columns if
            el not in td014_raw_cols + ['score_gen_ecs_lib_infer', 'gen_ecs_concat_txt_desc']]
    cols.append('td014_generateur_ecs_id')
    td014_p = td014[cols]

    td001_sys_ecs_agg = agg_systeme_ecs_essential(td001, td013, td014)

    return td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg


def build_doc(annexe_dir):
    from doc_annexe import td001_annexe_enveloppe_agg_desc, td001_sys_ch_agg_desc, td001_sys_ecs_agg_desc, \
        td007_annexe_desc, td008_annexe_desc, td012_annexe_desc, td014_annexe_desc, enums_cstb,td001_annexe_generale_desc

    doc_annexe = dict()
    doc_annexe['td001_annexe_generale'] = td001_annexe_generale_desc
    doc_annexe['td001_annexe_enveloppe_agg'] = td001_annexe_enveloppe_agg_desc
    doc_annexe['td001_sys_ch_agg'] = td001_sys_ch_agg_desc
    doc_annexe['td001_sys_ecs_agg'] = td001_sys_ecs_agg_desc
    doc_annexe['td007_annexe'] = td007_annexe_desc
    doc_annexe['td008_annexe'] = td008_annexe_desc
    doc_annexe['td012_annexe'] = td012_annexe_desc
    doc_annexe['td014_annexe'] = td014_annexe_desc

    with open(annexe_dir / 'doc_table_annexes_cstb.json', 'w', encoding='utf-8') as f:
        json.dump(doc_annexe, f, indent=4)

    with open(annexe_dir / 'enum_table_annexes_cstb.json', 'w', encoding='utf-8') as f:
        json.dump(enums_cstb, f, indent=4)


if __name__ == '__main__':

    data_dir = r'D:\data\dpe_full\depts'
    annexe_dir = r'D:\data\dpe_full\annexes_cstb'
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    build_doc(annexe_dir)
    for dept_dir in Path(data_dir).iterdir():
        print(dept_dir)
        annexe_dept_dir = annexe_dir / dept_dir.name
        annexe_dept_dir.mkdir(exist_ok=True, parents=True)
        # LOAD TABLES
        td007 = pd.read_csv(dept_dir / 'td007_paroi_opaque.csv', dtype=str)
        td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
        td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
        td008 = pd.read_csv(dept_dir / 'td008_baie.csv', dtype=str)
        td008 = td008.drop('td008_baie_id', axis=1)

        # ENVELOPPE PROCESSING
        td001_enveloppe_agg, td008_p, td007_p = run_enveloppe_processing(td001, td006, td007, td008)

        round_float_cols(td001_enveloppe_agg).to_csv(annexe_dept_dir / 'td001_annexe_enveloppe_agg.csv')
        round_float_cols(td007_p).to_csv(annexe_dept_dir / 'td007_paroi_opaque_annexe.csv')
        round_float_cols(td008_p).to_csv(annexe_dept_dir / 'td008_baie_annexe.csv')

        # SYSTEM PROCESSING

        td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
        td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
        td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
        td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

        td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                        td011, td012,
                                                                                                        td013, td014)
        round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_annexe_sys_ch_agg.csv')
        round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_annexe_sys_ecs_agg.csv')
        round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_annexe_installation_chauffage.csv')
        round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_annexe_generateur_chauffage.csv')
        round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_annexe_installation_ecs.csv')
        round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_annexe_generateur_ecs.csv')

        # add td001 processing
        postprocessing_td001(td001)[['nom_methode_dpe_norm', 'id']].rename(columns={'id': 'td001_dpe_id'}).to_csv(
            annexe_dept_dir / 'td001_annexe_generale.csv')
        break

D:\data\dpe_full\depts\1
tv013_valeur_pont_thermique_isolation_planche_bas not found
tv025_type_batiment not found
tv025_type_emetteur not found
tv025_equipement_intermittence not found
tv026_classe_inertie_plancher_bas not found
tv026_classe_inertie_plancher_haut not found
tv026_classe_inertie_paroi_verticale not found
tv026_classe_inertie_classe_inertie not found
tv029_simu_type_distribution not found


D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\utils.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[pond_col] = table[pond]
D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\utils.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[pond_value_col_temp] = table[pond_col] * table[value_col]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value 

In [15]:
table = td001_enveloppe_agg

In [5]:
td001_enveloppe_agg.Ubaie_avg.describe()

count    42587.000000
mean         2.470130
std          0.556546
min          0.100000
25%          2.200000
50%          2.423000
75%          2.700000
max          6.200000
Name: Ubaie_avg, dtype: float64

In [20]:
table

,Umurs_ext_avg,Umurs_deper_avg,Uplancher_bas_deper_avg,Uplancher_haut_deper_avg,is_plancher_bas_deper_isole,is_plancher_haut_deper_isole,is_murs_ext_isole,is_murs_deper_isole,mat_murs_deper_top,mat_murs_ext_top,...,surfaces_vitree_orientee_est,surfaces_vitree_orientee_est_ou_ouest,surfaces_vitree_orientee_horizontale,surfaces_vitree_orientee_nondef,surfaces_vitree_orientee_nord,surfaces_vitree_orientee_ouest,surfaces_vitree_orientee_sud,ratio_surface_vitree_exterieur,ratio_surface_vitree_deperditif,ratio_surface_vitree_total
td001_dpe_id,,,,,,,,,,,,,,,,,,,,,
1000142,NaN,0.71,0.70,0.15,Oui,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,1.44,NaN,NaN,NaN,2.46,2.88,6.72,NaN,0.11,0.11
1000989,1.75,1.75,1.42,0.21,Non,Oui,Non,Non,NONDEF,NONDEF,...,NaN,NaN,NaN,NaN,NaN,5.05,5.21,0.12,0.12,0.12
1001934,0.45,0.45,0.42,0.25,Terre Plein,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,NaN,NaN,NaN,NaN,NaN,1.87,10.42,0.16,0.15,0.15
1002065,0.70,0.70,NaN,NaN,NaN,NaN,Oui,Oui,Murs en béton banché,Murs en béton banché,...,12.43,NaN,NaN,NaN,3.74,NaN,NaN,NaN,0.79,0.79
100207,0.30,0.30,0.59,0.23,Oui,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,1.42,NaN,NaN,NaN,8.30,0.37,8.64,0.16,0.15,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968172,NaN,NaN,2.00,2.50,Non,Non,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986070,NaN,NaN,NaN,NaN,Non,Oui,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987387,NaN,NaN,NaN,NaN,Non,Oui,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
td007_annexe_desc={'materiaux_structure':"matériau de structure du composant d'enveloppe considéré au sens des tables valeurs tv004,tv006,tv008",
 'is_custom_resistance_thermique_isolation':"booleen si vrai : alors le diagnostiqueur a rentré une resistance thermique d'isolation et n'est pas passé par la table tv003/tv005/tv007",
 'is_custom_epaisseur_isolation':"booleen si vrai : alors le diagnostiqueur a rentré une epaisseur d'isolant thermique et n'est pas passé par la table tv003/tv005/tv007",
 'resistance_thermique_isolation_calc':"resistance thermique d'isolation recalculée a partir du coefficient de deperditions final et non isolé",
 'epaisseur_isolation_calc':"epaisseur d'isolation thermique recalculée a partir du coefficient de deperditions final et non isolé (en prenant hypothèse lambda =0.04)",
 'epaisseur_isolation_glob':"variable synthèse de epaisseur_isolation et epaisseur_isolation_calc",
 'resistance_thermique_isolation_glob':"variable synthèse de resistance_thermique_isolation et resistance_thermique_isolation_calc",
 'is_paroi_isole':'booleen qui dit si la paroi est isolée ou non (epaisseur isolation >=5cm)',
 'surface_baie_sum':'somme de la surface des baie',
 'surfacexnb_baie_calc_sum':'donnée redressée de la surface de baie (surface x nombre_baie)',
 'nb_baie_calc':'nombre de baies recalculée',
'surface_paroi_opaque_infer':'surface de paroi opaque calculé à partir de surface_paroi et des surfaces de baies',
 'surface_paroi_opaque_deperditive_infer':'surface de paroi opaque deperditive calculé à partir de surface_paroi et des surfaces de baies et du coefficient de reduction des deperditions.(b>0)',
 'b_infer':'coefficient b de réduction des deperditions déduit des tables tv.',
 'surface_paroi_opaque_exterieur_infer':'surface de paroi opaque donnant sur exterieur calculé à partir de surface_paroi et des surfaces de baies et du coefficient de reduction des deperditions.(b>0.95)',}

In [58]:
postprocessing_td001(td001)['nom_methode_dpe_norm'].value_counts().index

Index(['FACTURE', '3CL 2012', 'THBCE(RT2012)/THC(RT2005)', 'DPE vierge',
       '3CL 2005', 'NON DEFINI'],
      dtype='object')

In [55]:
td001_enveloppe_agg.is_plancher_bas_deper_isole.value_counts().index

Index(['Terre Plein', 'Oui', 'Non'], dtype='object')

In [47]:
td008_p.cat_baie_simple_infer.value_counts().index.categories

Index(['baie vitrée', 'paroi en brique de verre ou polycarbonate', 'porte'], dtype='object')

In [51]:
td008_p.type_vitrage_simple_infer.value_counts()

double vitrage                      336736
porte                                49682
simple vitrage                       19648
triple vitrage                        2303
NONDEF                                1329
INCOHERENT                            1003
brique de verre ou polycarbonate       753
Name: type_vitrage_simple_infer, dtype: int64

In [9]:
td008_p

NameError: name 'td008_p' is not defined

In [34]:
td014_p.columns.tolist()

['td014_generateur_ecs_id',
 'td001_dpe_id',
 'tr004_code',
 'tr004_description',
 'tv045_code',
 'tv045_energie',
 'tv045_methode',
 'tv045_type_production',
 'tv045_taux_conversion',
 'tv045_conversion_pci_pcs',
 'tv046_code',
 'tv046_nom_reseau',
 'tv046_localisation',
 'tv046_co2',
 'tv046_date_debut',
 'tv046_date_fin',
 'tv016_code',
 'tv016_departement',
 'tv016_altmin',
 'tv016_altmax',
 'tv016_nref',
 'tv016_dhref',
 'tv016_pref',
 'tv016_c2',
 'tv016_c3',
 'tv016_c4',
 'tv016_t_ext_basse',
 'tv016_e',
 'tv016_fch',
 'tv016_fecs_ancienne_m_i',
 'tv016_fecs_recente_m_i',
 'tv016_fecs_solaire_m_i',
 'tv016_fecs_ancienne_i_c',
 'tv016_fecs_recente_i_c',
 'tv017_code',
 'tv017_t_ext_moyen',
 'tv017_peta_cw',
 'tv017_dh14',
 'tv017_prs1',
 'tv018_code',
 'tv018_sclim_inf_150',
 'tv018_sclim_sup_150',
 'tv018_rclim_autres_etages',
 'tv018_rclim_dernier_etage',
 'tv046_type_production',
 'tv032_code',
 'tv032_type_generateur',
 'tv032_cop',
 'tv036_code',
 'tv036_type_generation',
 '

In [13]:
td012_p.type_energie_chauffage.value_counts().index

CategoricalIndex(['Electricité', 'Gaz', 'Bois, Biomasse', 'Autres énergies',
                  'Production d'électricité'],
                 categories=['Autres énergies', 'Bois, Biomasse', 'Electricité', 'Gaz', 'Production d'électricité'], ordered=False, dtype='category')

In [37]:
from td011_td012_processing import gen_ch_normalized_lib_matching_dict,gen_ch_lib_simp_dict

 from td013_td014_processing import gen_ecs_normalized_lib_matching_dict,gen_ecs_lib_simp_dict

enums = {
    'gen_ch_lib_infer':list(gen_ch_normalized_lib_matching_dict.keys()),
    'gen_ch_lib_infer_simp':list(gen_ch_lib_simp_dict.keys()),
    'type_energie_chauffage':['Electricité', 'Gaz', 'Bois, Biomasse', 'Autres énergies'],
    'type_installation_chauffage':['Chauffage Individuel', 'Chauffage Collectif'],
    'configuration_sys_chauffage':['type de générateur unique/installation unique',
       'types de générateur multiples/installations multiples',
       'types de générateur multiples/installation unique'],
}

IndentationError: unexpected indent (<ipython-input-37-300f404108e3>, line 3)

In [26]:
td001_sys_ch_agg.columns.tolist()

['td001_dpe_id',
 'nombre_installations_ch_total',
 'nombre_generateurs_ch_total',
 'mix_energetique_chauffage',
 'type_installation_chauffage_concat',
 'gen_ch_lib_infer_concat',
 'gen_ch_lib_infer_simp_concat',
 'configuration_sys_chauffage',
 'sys_ch_principal_consommation_chauffage',
 'sys_ch_principal_gen_ch_lib_infer',
 'sys_ch_principal_gen_ch_lib_infer_simp',
 'sys_ch_principal_nb_generateur',
 'sys_ch_principal_type_energie_chauffage',
 'sys_ch_principal_type_installation_chauffage',
 'sys_ch_secondaire_consommation_chauffage',
 'sys_ch_secondaire_gen_ch_lib_infer',
 'sys_ch_secondaire_gen_ch_lib_infer_simp',
 'sys_ch_secondaire_nb_generateur',
 'sys_ch_secondaire_type_energie_chauffage',
 'sys_ch_secondaire_type_installation_chauffage',
 'sys_ch_tertiaire_consommation_chauffage',
 'sys_ch_tertiaire_gen_ch_lib_infer_concat',
 'sys_ch_tertiaire_gen_ch_lib_infer_simp_concat',
 'sys_ch_tertiaire_nb_generateurs',
 'sys_ch_tertiaire_type_energie_chauffage_concat',
 'sys_ch_tertiair

In [25]:
td001_sys_ecs_agg.columns.tolist()

['td001_dpe_id',
 'nombre_installations_ecs_total',
 'nombre_generateurs_ecs_total',
 'mix_energetique_ecs',
 'type_installation_ecs_concat',
 'gen_ecs_lib_infer_concat',
 'gen_ecs_lib_infer_simp_concat',
 'configuration_sys_ecs',
 'sys_ecs_principal_gen_ecs_lib_infer',
 'sys_ecs_principal_gen_ecs_lib_infer_simp',
 'sys_ecs_principal_nb_generateur',
 'sys_ecs_principal_type_energie_ecs',
 'sys_ecs_principal_type_installation_ecs',
 'sys_ecs_secondaire_gen_ecs_lib_infer',
 'sys_ecs_secondaire_gen_ecs_lib_infer_simp',
 'sys_ecs_secondaire_nb_generateur',
 'sys_ecs_secondaire_type_energie_ecs',
 'sys_ecs_secondaire_type_installation_ecs',
 'sys_ecs_tertiaire_gen_ecs_lib_infer_concat',
 'sys_ecs_tertiaire_gen_ecs_lib_infer_simp_concat',
 'sys_ecs_tertiaire_nb_generateurs',
 'sys_ecs_tertiaire_type_energie_ecs_concat',
 'sys_ecs_tertiaire_type_installation_ecs_concat']

In [23]:
td001_sys_ecs_agg.columns.value_counts().index

AttributeError: 'DataFrame' object has no attribute 'configuration_sys_chauffage'

In [27]:
td001_sys_ch_agg_desc = { 
    'td001_dpe_id': 'identifiant de la table td001_dpe_id',
'nombre_installations_ch_total':"nombre d'installations totale de chauffage pour le dpe.",
 'nombre_generateurs_ch_total':'nombre de générateurs de chauffage totaux pour le dpe ',
 'mix_energetique_chauffage':"type d'energie de chauffage concaténé  (voir enumérateur type_energie_chauffage)",
 'type_installation_chauffage_concat':"type d'installation de chauffage concaténé (collectif ou individuel)",
 'gen_ch_lib_infer_concat':'concaténation du libéllé générateur chauffage (voir enumerateur : gen_ch_lib_infer)',
 'gen_ch_lib_infer_simp_concat':"libéllé générateur chauffage simplifié concaténé (voir enumerateur : gen_ch_lib_infer_simp)",
 'configuration_sys_chauffage':'configuration des systèmes de chauffage pour le DPE (voir enumerateur : configuration_sys_chauffage)',
 'sys_ch_principal_consommation_chauffage':'consommation de chauffage du système de chauffage principal',
 'sys_ch_principal_gen_ch_lib_infer':'libéllé générateur chauffage pour le système de chauffage principal (voir enumerateur : gen_ch_lib_infer)',
 'sys_ch_principal_gen_ch_lib_infer_simp':'libéllé générateur chauffage simplifié pour le système de chauffage principal (voir enumerateur : gen_ch_lib_infer_simp)',
 'sys_ch_principal_nb_generateur':'nombre de générateurs correspondant au système de chauffage principal (si plusieurs chaudières de même type ou plusieurs emetteurs effet joule ils sont regroupés)',
 'sys_ch_principal_type_energie_chauffage':"type d'energie de chauffage dy système de chauffage principal  (voir enumérateur type_energie_chauffage)",
 'sys_ch_principal_type_installation_chauffage':"type d'installation de chauffage pour le système de chauffage principal (collectif ou individuel)",
 'sys_ch_secondaire_consommation_chauffage':'consommation de chauffage du système de chauffage secondaire',
 'sys_ch_secondaire_gen_ch_lib_infer':'libéllé générateur chauffage pour le système de chauffage secondaire (voir enumerateur : gen_ch_lib_infer)',
 'sys_ch_secondaire_gen_ch_lib_infer_simp':'libéllé générateur chauffage simplifié pour le système de chauffage secondaire (voir enumerateur : gen_ch_lib_infer_simp)',
 'sys_ch_secondaire_nb_generateur':'nombre de générateurs correspondant au système de chauffage secondaire (si plusieurs chaudières de même type ou plusieurs emetteurs effet joule ils sont regroupés)',
 'sys_ch_secondaire_type_energie_chauffage':"type d'energie de chauffage dy système de chauffage secondaire  (voir enumérateur type_energie_chauffage)",
 'sys_ch_secondaire_type_installation_chauffage':"type d'installation de chauffage pour le système de chauffage secondaire (collectif ou individuel)",
 'sys_ch_tertiaire_consommation_chauffage_concat':'consommation de chauffage du système de chauffage tertiaire concaténé',
 'sys_ch_tertiaire_gen_ch_lib_infer_concat':'libéllé générateur chauffage pour le système de chauffage tertiaire concaténé (voir enumerateur : gen_ch_lib_infer)',
 'sys_ch_tertiaire_gen_ch_lib_infer_simp_concat':'libéllé générateur chauffage simplifié pour le système de chauffage tertiaire concaténé (voir enumerateur : gen_ch_lib_infer_simp)',
 'sys_ch_tertiaire_nb_generateur':'nombre de générateurs correspondant au système de chauffage tertiaire (tous types de générateurs confondus)',
 'sys_ch_tertiaire_type_energie_chauffage_concat':"type d'energie de chauffage du système de chauffage tertiaire concaténé  (voir enumérateur type_energie_chauffage)",
 'sys_ch_tertiaire_type_installation_chauffage_concat':"type d'installation de chauffage pour le système de chauffage tertiaire concaténé (collectif ou individuel)",
 }



td001_sys_ecs_agg_desc = { 
    'td001_dpe_id': 'identifiant de la table td001_dpe_id',
'nombre_installations_ecs_total':"nombre d'installations totale de ecs pour le dpe.",
 'nombre_generateurs_ecs_total':'nombre de générateurs de ecs totaux pour le dpe ',
 'mix_energetique_ecs':"type d'energie de ecs concaténé  (voir enumérateur type_energie_ecs)",
 'type_installation_ecs_concat':"type d'installation de ecs concaténé (collectif ou individuel)",
 'gen_ecs_lib_infer_concat':'concaténation du libéllé générateur ecs (voir enumerateur : gen_ecs_lib_infer)',
 'gen_ecs_lib_infer_simp_concat':"libéllé générateur ecs simplifié concaténé (voir enumerateur : gen_ecs_lib_infer_simp)",
 'configuration_sys_ecs':'configuration des systèmes de ecs pour le DPE (voir enumerateur : configuration_sys_ecs)',
 'sys_ecs_principal_consommation_ecs':'consommation de ecs du système de ecs principal',
 'sys_ecs_principal_gen_ecs_lib_infer':'libéllé générateur ecs pour le système de ecs principal (voir enumerateur : gen_ecs_lib_infer)',
 'sys_ecs_principal_gen_ecs_lib_infer_simp':'libéllé générateur ecs simplifié pour le système de ecs principal (voir enumerateur : gen_ecs_lib_infer_simp)',
 'sys_ecs_principal_nb_generateur':'nombre de générateurs correspondant au système de ecs principal (si plusieurs chaudières de même type ou plusieurs emetteurs effet joule ils sont regroupés)',
 'sys_ecs_principal_type_energie_ecs':"type d'energie de ecs dy système de ecs principal  (voir enumérateur type_energie_ecs)",
 'sys_ecs_principal_type_installation_ecs':"type d'installation de ecs pour le système de ecs principal (collectif ou individuel)",
 'sys_ecs_secondaire_consommation_ecs':'consommation de ecs du système de ecs secondaire',
 'sys_ecs_secondaire_gen_ecs_lib_infer':'libéllé générateur ecs pour le système de ecs secondaire (voir enumerateur : gen_ecs_lib_infer)',
 'sys_ecs_secondaire_gen_ecs_lib_infer_simp':'libéllé générateur ecs simplifié pour le système de ecs secondaire (voir enumerateur : gen_ecs_lib_infer_simp)',
 'sys_ecs_secondaire_nb_generateur':'nombre de générateurs correspondant au système de ecs secondaire (si plusieurs chaudières de même type ou plusieurs emetteurs effet joule ils sont regroupés)',
 'sys_ecs_secondaire_type_energie_ecs':"type d'energie de ecs dy système de ecs secondaire  (voir enumérateur type_energie_ecs)",
 'sys_ecs_secondaire_type_installation_ecs':"type d'installation de ecs pour le système de ecs secondaire (collectif ou individuel)",
 'sys_ecs_tertiaire_consommation_ecs_concat':'consommation de ecs du système de ecs tertiaire concaténé',
 'sys_ecs_tertiaire_gen_ecs_lib_infer_concat':'libéllé générateur ecs pour le système de ecs tertiaire concaténé (voir enumerateur : gen_ecs_lib_infer)',
 'sys_ecs_tertiaire_gen_ecs_lib_infer_simp_concat':'libéllé générateur ecs simplifié pour le système de ecs tertiaire concaténé (voir enumerateur : gen_ecs_lib_infer_simp)',
 'sys_ecs_tertiaire_nb_generateur':'nombre de générateurs correspondant au système de ecs tertiaire (tous types de générateurs confondus)',
 'sys_ecs_tertiaire_type_energie_ecs_concat':"type d'energie de ecs du système de ecs tertiaire concaténé  (voir enumérateur type_energie_ecs)",
 'sys_ecs_tertiaire_type_installation_ecs_concat':"type d'installation de ecs pour le système de ecs tertiaire concaténé (collectif ou individuel)",
 }



In [ ]:
'sys_ch_tertiaire_consommation_chauffage':'',
 'sys_ch_tertiaire_gen_ch_lib_infer_concat':'',
 'sys_ch_tertiaire_gen_ch_lib_infer_simp_concat':'',
 'sys_ch_tertiaire_nb_generateurs':'',
 'sys_ch_tertiaire_type_energie_chauffage_concat':'',
 'sys_ch_tertiaire_type_installation_chauffage_concat':'',

In [6]:
postprocessing_td001(td001)

,id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,...,surface_planchers_hauts_deperditifs,surface_planchers_bas_deperditifs,surface_parois_verticales_opaques_deperditives,etat_avancement,organisme_certificateur,adresse_organisme_certificateur,dpe_vierge,est_efface,date_reception_dpe,nom_methode_dpe_norm
0,178,1301V1000003T,6991,3,1,3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-10,...,0.00,0.00,0.00,1,DEKRA,3/5 avenue Garlande 92220 BAGNEUX (détail sur ...,0,0,2013-04-11 04:00:00,3CL 2012
1,390,1301V2000001S,8334,3,2,Méthode Facture,NaN,NaN,NaN,2013-04-15,...,0.00,0.00,0.00,1,Bureau VERITAS,Le Guillaumet 92046 PARIS LA DEFENSE CEDEX (dé...,0,0,2013-04-15 04:00:00,FACTURE
2,427,1301V1000006W,6931,7,1,3CL,Version 1.3,NaN,NaN,2013-04-08,...,102.38,102.38,96.37,1,Organisme certificateur,-,0,0,2013-04-15 04:00:00,3CL 2012
3,428,1301V1000007X,6931,7,1,3CL,Version 1.3,NaN,NaN,2013-04-05,...,114.80,114.95,121.02,1,Organisme certificateur,-,0,0,2013-04-15 04:00:00,3CL 2012
4,524,1301V1000008Y,8334,3,1,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-16,...,0.00,0.00,0.00,1,Bureau VERITAS,Le Guillaumet 92046 PARIS LA DEFENSE CEDEX (dé...,0,0,2013-04-16 04:00:00,3CL 2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97959,9430721,2001V1000848I,17527,3,1,Méthode 3CL,"3CL-DPE, version 1.3",NaN,NaN,2020-03-09,...,82.61,82.61,120.93,1,DEKRA Certification,3/5 avenue Garlande 92220 BAGNEUX,0,0,2020-03-10 11:56:02,3CL 2012
97960,9430783,2001V2000686J,20120,3,2,Méthode Facture,NaN,NaN,NaN,2020-03-10,...,0.00,0.00,0.00,1,AFNOR Certification,11 rue Francis de Pressensé 93571 SAINT DENIS...,0,0,2020-03-10 12:03:21,FACTURE
97961,9431090,2001V4000024X,16555,6,4,Factures,2012,NaN,NaN,2020-03-10,...,40.10,40.10,0.00,1,ITGA,test,1,0,2020-03-10 12:52:07,FACTURE
97962,9431176,2001L1000271X,829,10,9,3CL - DPE,V2012,NaN,NaN,2020-03-10,...,NaN,NaN,NaN,1,SOCOTEC Certification France,"1, rue René Anjolvy\n94250 GENTILLY",NaN,0,2020-03-10 13:08:33,3CL 2012


In [ ]:
td001_sys_ch_agg_desc = { 'td001_dpe_id': 'identifiant de la table td001_dpe_id',
'nombre_installations_ch_total':"nombre d'installations totale de chauffage pour le dpe.",
 'nombre_generateurs_ch_total':'nombre de générateurs de chauffage totaux pour le dpe ',
 'mix_energetique_chauffage':"type d'energie de chauffage concaténé  (voir enumérateur type_energie_chauffage)",
 'type_installation_chauffage_concat':"type d'installation de chauffage concaténé (collectif ou individuel)",
 'gen_ch_lib_infer_concat':'concaténation du libéllé générateur chauffage (voir enumerateur : gen_ch_lib_infer)',
 'gen_ch_lib_infer_simp_concat':"libéllé générateur chauffage simplifié concaténé (voir enumerateur : gen_ch_lib_infer_simp)",
 'configuration_sys_chauffage':'configuration des systèmes de chauffage pour le DPE (voir enumerateur : configuration_sys_chauffage)',
 'sys_ch_principal_consommation_chauffage':'consommation de chauffage du système de chauffage principal',
 'sys_ch_principal_gen_ch_lib_infer':'libéllé générateur chauffage pour le système de chauffage principal (voir enumerateur : gen_ch_lib_infer)',
 'sys_ch_principal_gen_ch_lib_infer_simp':'libéllé générateur chauffage simplifié pour le système de chauffage principal (voir enumerateur : gen_ch_lib_infer_simp)',
 'sys_ch_principal_nb_generateur':'nombre de générateurs correspondant au système de chauffage principal (si plusieurs chaudières de même type ou plusieurs emetteurs effet joule ils sont regroupés)',
 'sys_ch_principal_type_energie_chauffage':"type d'energie de chauffage dy système de chauffage principal  (voir enumérateur type_energie_chauffage)",
 'sys_ch_principal_type_installation_chauffage':"type d'installation de chauffage pour le système de chauffage principal (collectif ou individuel)",
 'sys_ch_secondaire_consommation_chauffage':'consommation de chauffage du système de chauffage secondaire',
 'sys_ch_secondaire_gen_ch_lib_infer':'libéllé générateur chauffage pour le système de chauffage secondaire (voir enumerateur : gen_ch_lib_infer)',
 'sys_ch_secondaire_gen_ch_lib_infer_simp':'libéllé générateur chauffage simplifié pour le système de chauffage secondaire (voir enumerateur : gen_ch_lib_infer_simp)',
 'sys_ch_secondaire_nb_generateur':'nombre de générateurs correspondant au système de chauffage secondaire (si plusieurs chaudières de même type ou plusieurs emetteurs effet joule ils sont regroupés)',
 'sys_ch_secondaire_type_energie_chauffage':"type d'energie de chauffage dy système de chauffage secondaire  (voir enumérateur type_energie_chauffage)",
 'sys_ch_secondaire_type_installation_chauffage':"type d'installation de chauffage pour le système de chauffage secondaire (collectif ou individuel)",
 'sys_ch_tertiaires concaténé_consommation_chauffage_concat':'consommation de chauffage du système de chauffage tertiaires concaténé',
 'sys_ch_tertiaires concaténé_gen_ch_lib_infer_concat':'libéllé générateur chauffage pour le système de chauffage tertiaires concaténé (voir enumerateur : gen_ch_lib_infer)',
 'sys_ch_tertiaires concaténé_gen_ch_lib_infer_simp_concat':'libéllé générateur chauffage simplifié pour le système de chauffage tertiaires concaténé (voir enumerateur : gen_ch_lib_infer_simp)',
 'sys_ch_tertiaires concaténé_nb_generateur':'nombre de générateurs correspondant au système de chauffage tertiaires (tous types de générateurs confondus)',
 'sys_ch_tertiaires concaténé_type_energie_chauffage_concat':"type d'energie de chauffage du système de chauffage tertiaires concaténé  (voir enumérateur type_energie_chauffage)",
 'sys_ch_tertiaires concaténé_type_installation_chauffage_concat':"type d'installation de chauffage pour le système de chauffage tertiaires concaténé (collectif ou individuel)",
 }


KeyError: "None of [Index(['nom_dpe_methode_norm', 'td001_dpe_id'], dtype='object')] are in the [columns]"

In [66]:
td001_enveloppe_agg

,Umurs_ext_avg,Umurs_deper_avg,Uplancher_bas_deper_avg,Uplancher_haut_deper_avg,is_plancher_bas_deper_isole,is_plancher_haut_deper_isole,is_murs_ext_isole,is_murs_deper_isole,mat_murs_deper_top,mat_murs_ext_top,...,surfaces_vitree_orientee_est,surfaces_vitree_orientee_est_ou_ouest,surfaces_vitree_orientee_horizontale,surfaces_vitree_orientee_nondef,surfaces_vitree_orientee_nord,surfaces_vitree_orientee_ouest,surfaces_vitree_orientee_sud,ratio_surface_vitree_exterieur,ratio_surface_vitree_deperditif,ratio_surface_vitree_total
td001_dpe_id,,,,,,,,,,,,,,,,,,,,,
1000142,NaN,0.706614,0.700000,0.150000,Oui,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,1.44,NaN,NaN,NaN,2.46,2.88,6.72,NaN,0.106191,0.106191
1000989,1.75,1.750000,1.417647,0.210000,Non,Oui,Non,Non,NONDEF,NONDEF,...,NaN,NaN,NaN,NaN,NaN,5.05,5.21,0.119455,0.119455,0.119455
1001934,0.45,0.450000,0.415763,0.250000,Terre Plein,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,NaN,NaN,NaN,NaN,NaN,1.87,10.42,0.158132,0.145754,0.145754
1002065,0.70,0.700000,NaN,NaN,NaN,NaN,Oui,Oui,Murs en béton banché,Murs en béton banché,...,12.43,NaN,NaN,NaN,3.74,NaN,NaN,NaN,0.785714,0.785714
100207,0.30,0.303571,0.590000,0.232461,Oui,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,1.42,NaN,NaN,NaN,8.30,0.37,8.64,0.155953,0.146672,0.146672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968172,NaN,NaN,2.000000,2.500000,Non,Non,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986070,NaN,NaN,NaN,NaN,Non,Oui,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987387,NaN,NaN,NaN,NaN,Non,Oui,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
td001_enveloppe_agg.mat_murs_ext_top.value_counts()

Murs en blocs de béton creux                                          18322
Murs en béton banché                                                   9841
NONDEF                                                                 7568
Inconnu                                                                1628
Murs en blocs de béton pleins                                          1262
Murs en briques creuses                                                1013
Murs en briques pleines simples                                         901
Murs en pierre de taille et moellons avec remplissage tout venant       770
Murs en pan de bois sans remplissage tout venant                        760
Murs en pierre de taille et moellons constitué d'un seul matériaux      709
Murs en briques pleines doubles avec lame d'air                         639
Murs en pisé ou béton de terre stabilisé (à partir d'argile crue)       451
Béton cellulaire                                                        347
Cloison de p

In [19]:
td001_enveloppe_agg.mat_murs_deper_top.value_counts()

Murs en blocs de béton creux                                          18252
Murs en béton banché                                                  10357
NONDEF                                                                 7388
Inconnu                                                                1459
Murs en blocs de béton pleins                                          1338
Murs en briques creuses                                                1003
Murs en briques pleines simples                                         903
Murs en pierre de taille et moellons avec remplissage tout venant       828
Murs en pan de bois sans remplissage tout venant                        750
Murs en pierre de taille et moellons constitué d'un seul matériaux      704
Murs en briques pleines doubles avec lame d'air                         641
Murs en pisé ou béton de terre stabilisé (à partir d'argile crue)       481
Murs en béton de mâchefer                                               329
Béton cellul

In [21]:
td001_enveloppe_agg.is_murs_deper_isole.value_counts()/(td001_enveloppe_agg.is_murs_deper_isole.isnull()==0).sum()

Oui    0.820662
Non    0.179338
Name: is_murs_deper_isole, dtype: float64

In [22]:
td001_enveloppe_agg.is_murs_ext_isole.value_counts()/(td001_enveloppe_agg.is_murs_deper_isole.isnull()==0).sum()

Oui    0.787465
Non    0.212535
Name: is_murs_ext_isole, dtype: float64

In [23]:
td001_enveloppe_agg.is_plancher_haut_deper_isole.value_counts()/(td001_enveloppe_agg.is_murs_deper_isole.isnull()==0).sum()

Oui    0.701428
Non    0.144015
Name: is_plancher_haut_deper_isole, dtype: float64

In [24]:
td001_enveloppe_agg.is_plancher_bas_deper_isole.value_counts()/(td001_enveloppe_agg.is_murs_deper_isole.isnull()==0).sum()

Terre Plein    0.308316
Oui            0.288495
Non            0.209789
Name: is_plancher_bas_deper_isole, dtype: float64

In [51]:
td001_enveloppe_agg

,Umurs_ext_avg,Umurs_deper_avg,Uplancher_bas_deper_avg,Uplancher_haut_deper_avg,is_plancher_bas_deper_isole,is_plancher_haut_deper_isole,is_murs_ext_isole,is_murs_deper_isole,mat_murs_deper_top,mat_murs_ext_top,...,surfaces_vitree_orientee_est,surfaces_vitree_orientee_est_ou_ouest,surfaces_vitree_orientee_horizontale,surfaces_vitree_orientee_nondef,surfaces_vitree_orientee_nord,surfaces_vitree_orientee_ouest,surfaces_vitree_orientee_sud,ratio_surface_vitree_exterieur,ratio_surface_vitree_deperditif,ratio_surface_vitree_total
td001_dpe_id,,,,,,,,,,,,,,,,,,,,,
1000142,NaN,0.706614,0.700000,0.150000,Oui,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,1.44,NaN,NaN,NaN,2.46,2.88,6.72,NaN,0.106191,0.106191
1000989,1.75,1.750000,1.417647,0.210000,Non,Oui,Non,Non,NONDEF,NONDEF,...,NaN,NaN,NaN,NaN,NaN,5.05,5.21,0.119455,0.119455,0.119455
1001934,0.45,0.450000,0.415763,0.250000,Terre Plein,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,NaN,NaN,NaN,NaN,NaN,1.87,10.42,0.158132,0.145754,0.145754
1002065,0.70,0.700000,NaN,NaN,NaN,NaN,Oui,Oui,Murs en béton banché,Murs en béton banché,...,12.43,NaN,NaN,NaN,3.74,NaN,NaN,NaN,0.785714,0.785714
100207,0.30,0.303571,0.590000,0.232461,Oui,Oui,Oui,Oui,Murs en blocs de béton creux,Murs en blocs de béton creux,...,1.42,NaN,NaN,NaN,8.30,0.37,8.64,0.155953,0.146672,0.146672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968172,NaN,NaN,2.000000,2.500000,Non,Non,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986070,NaN,NaN,NaN,NaN,Non,Oui,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987387,NaN,NaN,NaN,NaN,Non,Oui,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
td001_enveloppe_agg.type_vitrage_simple_top.value_counts()

double vitrage                      40954
simple vitrage                       1569
triple vitrage                        226
INCOHERENT                            102
NONDEF                                 91
brique de verre ou polycarbonate       10
porte                                   0
Name: type_vitrage_simple_top, dtype: int64

In [42]:
td001_enveloppe_agg.columns.sort_values()

Index(['Ubaie_avg', 'Umurs_deper_avg', 'Umurs_ext_avg',
       'Uplancher_bas_deper_avg', 'Uplancher_haut_deper_avg',
       'is_murs_deper_isole', 'is_murs_ext_isole',
       'is_plancher_bas_deper_isole', 'is_plancher_haut_deper_isole',
       'mat_murs_deper_top', 'mat_murs_ext_top', 'mat_plancher_bas_deper_top',
       'mat_plancher_haut_deper_top', 'orientation_concat',
       'ratio_surface_vitree_deperditif', 'ratio_surface_vitree_exterieur',
       'ratio_surface_vitree_total', 'surf_murs_deper', 'surf_murs_ext',
       'surf_murs_totale', 'surf_pb_deper', 'surf_pb_totale', 'surf_ph_deper',
       'surf_ph_totale', 'surface_porte_totale', 'surface_vitree_totale',
       'surfaces_vitree_orientee_est', 'surfaces_vitree_orientee_est_ou_ouest',
       'surfaces_vitree_orientee_horizontale',
       'surfaces_vitree_orientee_nondef', 'surfaces_vitree_orientee_nord',
       'surfaces_vitree_orientee_ouest', 'surfaces_vitree_orientee_sud',
       'type_vitrage_simple_top'],
      dtyp

In [52]:
 td001_enveloppe_agg[[  'surfaces_vitree_orientee_est', 'surfaces_vitree_orientee_est_ou_ouest',
       'surfaces_vitree_orientee_horizontale',
       'surfaces_vitree_orientee_nondef', 'surfaces_vitree_orientee_nord',
       'surfaces_vitree_orientee_ouest', 'surfaces_vitree_orientee_sud',] ]

,surfaces_vitree_orientee_est,surfaces_vitree_orientee_est_ou_ouest,surfaces_vitree_orientee_horizontale,surfaces_vitree_orientee_nondef,surfaces_vitree_orientee_nord,surfaces_vitree_orientee_ouest,surfaces_vitree_orientee_sud
td001_dpe_id,,,,,,,
1000142,1.44,NaN,NaN,NaN,2.46,2.88,6.72
1000989,NaN,NaN,NaN,NaN,NaN,5.05,5.21
1001934,NaN,NaN,NaN,NaN,NaN,1.87,10.42
1002065,12.43,NaN,NaN,NaN,3.74,NaN,NaN
100207,1.42,NaN,NaN,NaN,8.30,0.37,8.64
...,...,...,...,...,...,...,...
968172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986070,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987387,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
td001_enveloppe_agg[['surf_murs_totale', 'surface_porte_totale', 'surface_vitree_totale',]]

,surf_murs_totale,surface_porte_totale,surface_vitree_totale
td001_dpe_id,,,
1000142,127.13,6.63,13.50
1000989,85.89,8.85,10.26
1001934,84.32,3.40,12.29
1002065,20.58,2.00,16.17
100207,127.70,5.50,18.73
...,...,...,...
968172,NaN,NaN,NaN
986070,NaN,NaN,NaN
987387,NaN,NaN,NaN


In [2]:
td001_enveloppe_agg['orientation_concat'].value_counts()

Ouest + Est                          5371
Ouest + Sud + Est                    5304
Nord + Ouest + Sud + Est             4335
Nord + Sud                           2960
Sud + Nord                           2959
                                     ... 
Horizontale + Ouest + Est               1
Horizontale + Sud + Est                 1
Nord + Sud + Horizontale                1
NONDEF + Horizontale + Est              1
Est ou Ouest + Nord + Est + Ouest       1
Name: orientation_concat, Length: 80, dtype: int64

In [ ]:
td001_sys_ch_agg

In [55]:
td001_sys_ch_agg.gen_ch_lib_infer_simp_concat.value_counts().to_dict()

{'generateurs a effet joule': 13295,
 'chaudiere gaz performante(condensation ou basse temperature)': 7518,
 'chaudiere gaz non déterminee': 4447,
 'generateurs a effet joule + poele ou insert bois': 3915,
 'chaudiere gaz standard': 3268,
 'chaudiere fioul non déterminee': 1593,
 'chaudiere fioul standard': 1077,
 'pac air/eau': 710,
 'chaudiere fioul non déterminee + poele ou insert bois': 453,
 'chaudiere gaz non déterminee + poele ou insert bois': 439,
 'chaudiere gaz performante(condensation ou basse temperature) + poele ou insert bois': 405,
 'chaudiere electrique': 371,
 'chaudiere fioul performante(condensation ou basse temperature)': 348,
 'poele ou insert bois': 341,
 'chaudiere fioul standard + poele ou insert bois': 280,
 'pac air/air': 263,
 'generateurs a effet joule + pac air/air': 253,
 'generateurs a effet joule + pac air/eau': 234,
 'chaudiere gaz standard + poele ou insert bois': 208,
 'pac indeterminee': 205,
 'reseau de chaleur': 194,
 'non affecte': 190,
 'generate

In [67]:
td001_sys_ch_agg.columns.tolist()

['td001_dpe_id',
 'nombre_installations',
 'nombre_generateur_total',
 'mix_energetique_chauffage',
 'type_installation_chauffage_concat',
 'gen_ch_lib_infer_concat',
 'gen_ch_lib_infer_simp_concat',
 'configuration_sys_chauffage',
 'sys_ch_principal_consommation_chauffage',
 'sys_ch_principal_gen_ch_lib_infer',
 'sys_ch_principal_gen_ch_lib_infer_simp',
 'sys_ch_principal_nb_generateur',
 'sys_ch_principal_type_energie_chauffage',
 'sys_ch_principal_type_installation_chauffage',
 'sys_ch_secondaire_consommation_chauffage',
 'sys_ch_secondaire_gen_ch_lib_infer',
 'sys_ch_secondaire_gen_ch_lib_infer_simp',
 'sys_ch_secondaire_nb_generateur',
 'sys_ch_secondaire_type_energie_chauffage',
 'sys_ch_secondaire_type_installation_chauffage',
 'sys_ch_tertiaire_consommation_chauffage',
 'sys_ch_tertiaire_gen_ch_lib_infer_concat',
 'sys_ch_tertiaire_gen_ch_lib_infer_simp_concat',
 'sys_ch_tertiaire_nb_generateurs',
 'sys_ch_tertiaire_type_energie_chauffage_concat',
 'sys_ch_tertiaire_type_instal

In [ ]:
 'nombre_installations':
 'nombre_generateur_total':
 'mix_energetique_chauffage':
 'type_installation_chauffage_concat':
 'gen_ch_lib_infer_concat':
 'gen_ch_lib_infer_simp_concat':
 'configuration_sys_chauffage':
 'sys_ch_principal_consommation_chauffage':
 'sys_ch_principal_gen_ch_lib_infer':
 'sys_ch_principal_gen_ch_lib_infer_simp':
 'sys_ch_principal_nb_generateur':
 'sys_ch_principal_type_energie_chauffage':
 'sys_ch_principal_type_installation_chauffage':
 'sys_ch_secondaire_consommation_chauffage':
 'sys_ch_secondaire_gen_ch_lib_infer':
 'sys_ch_secondaire_gen_ch_lib_infer_simp':
 'sys_ch_secondaire_nb_generateur':
 'sys_ch_secondaire_type_energie_chauffage':
 'sys_ch_secondaire_type_installation_chauffage':
 'sys_ch_tertiaire_consommation_chauffage':
 'sys_ch_tertiaire_gen_ch_lib_infer_concat':
 'sys_ch_tertiaire_gen_ch_lib_infer_simp_concat':
 'sys_ch_tertiaire_nb_generateurs':
 'sys_ch_tertiaire_type_energie_chauffage_concat':
 'sys_ch_tertiaire_type_installation_chauffage_concat':

In [56]:
td001_sys_ch_agg.mix_energetique_chauffage.value_counts()

Electricité                                                                       15703
Gaz                                                                               14978
Bois, Biomasse + Electricité                                                       4700
Autres énergies                                                                    3708
Autres énergies + Bois, Biomasse                                                   1091
Bois, Biomasse + Gaz                                                               1008
Bois, Biomasse                                                                      545
Electricité + Gaz                                                                   428
Autres énergies + Electricité                                                       366
Autres énergies + Bois, Biomasse + Electricité                                      190
Bois, Biomasse + Electricité + Gaz                                                  107
Bois, Biomasse + Electricité + E

In [2]:
td001_sys_ecs_agg.gen_ecs_lib_infer_concat.value_counts()

ballon a accumulation electrique                                                                            23277
ECS thermodynamique electrique(PAC ou ballon)                                                                7807
chaudiere gaz                                                                                                6763
chaudiere mixte gaz                                                                                          6001
ecs solaire thermique + ballon a accumulation electrique                                                     3402
ecs solaire thermique + chaudiere mixte gaz                                                                  1932
chaudiere fioul                                                                                              1418
non affecte                                                                                                   946
chaudiere mixte fioul                                                                   

In [60]:
td001_sys_ecs_agg.mix_energetique_ecs.value_counts()

Electricité                                       31066
Gaz                                               17657
Autres énergies                                    4372
Autres énergies + Electricité                       337
Bois, Biomasse                                      270
Electricité + Gaz                                   230
Bois, Biomasse + Electricité                         56
Autres énergies + Bois, Biomasse                      7
Autres énergies + Bois, Biomasse + Electricité        5
Autres énergies + Gaz                                 2
Bois, Biomasse + Gaz                                  2
Name: mix_energetique_ecs, dtype: int64